In [1]:
import cv2

faces_xml = 'haarcascade_frontalface_alt.xml'
eyes_xml = 'haarcascade_eye_tree_eyeglasses.xml'
face_cascade = cv2.CascadeClassifier()
eyes_cascade = cv2.CascadeClassifier()

# -- 1. Load the cascades
if not face_cascade.load(faces_xml):
    print('--(!)Error loading face cascade')
    exit(0)
    
if not eyes_cascade.load(eyes_xml):
    print('--(!)Error loading eyes cascade')
    exit(0)
    
# -- 2. Read the video stream
camera = cv2.VideoCapture(0)
if not camera.isOpened:
    print('--(!)Error opening video capture')
    exit(0)
    
while True:
    ret, frame = camera.read()
    if frame is None:
        print('--(!) No captured frame -- Break!')
        break
    
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #frame_gray = cv2.equalizeHist(frame_gray)
    # -- Detect faces
    faces = face_cascade.detectMultiScale(frame_gray)
    for (x, y, w, h) in faces:
        center = (x + w // 2, y + h // 2)   
        faceROI = frame_gray[y:y + h, x:x + w]
        
        # -- In each face, detect eyes
        eyes = eyes_cascade.detectMultiScale(faceROI)
        for (x2, y2, w2, h2) in eyes:
            eye_center = (x + x2 + w2 // 2, y + y2 + h2 // 2)
            radius = int(round((w2 + h2) * 0.25))
            frame = cv2.circle(frame, eye_center, radius, (255, 0, 0), 4)
        frame = cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 255), 4)
    cv2.imshow('Capture - Face detection', frame)
    if cv2.waitKey(10) == 27:
        break